In [71]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

In [72]:
url = f"https://api.binance.com/api/v3/depth"
params = {
    "symbol": "SOLUSDT",
    "limit": 10
}

orderbook = requests.get(url, params=params).json()
orderbook

{'lastUpdateId': 26503634372,
 'bids': [['142.88000000', '370.87500000'],
  ['142.87000000', '285.05700000'],
  ['142.86000000', '667.03800000'],
  ['142.85000000', '644.68900000'],
  ['142.84000000', '1478.04800000'],
  ['142.83000000', '1477.73000000'],
  ['142.82000000', '556.19400000'],
  ['142.81000000', '253.64500000'],
  ['142.80000000', '290.85100000'],
  ['142.79000000', '378.69100000']],
 'asks': [['142.89000000', '122.42500000'],
  ['142.90000000', '667.18300000'],
  ['142.91000000', '289.49500000'],
  ['142.92000000', '141.42000000'],
  ['142.93000000', '82.80200000'],
  ['142.94000000', '160.84100000'],
  ['142.95000000', '124.37500000'],
  ['142.96000000', '108.83200000'],
  ['142.97000000', '152.22100000'],
  ['142.98000000', '238.42500000']]}

In [73]:
ask_levels = pd.DataFrame(orderbook['asks'], columns=['price', 'quantity'], dtype=float)
ask_levels['side'] = 'ask'
ask_levels = ask_levels.sort_values('price')
ask_levels

,price,quantity,side
0,142.89,122.425,ask
1,142.90,667.183,ask
2,142.91,289.495,ask
3,142.92,141.420,ask
4,142.93,82.802,ask
5,142.94,160.841,ask
6,142.95,124.375,ask
7,142.96,108.832,ask
8,142.97,152.221,ask
9,142.98,238.425,ask


In [74]:
bid_levels = pd.DataFrame(orderbook['bids'], columns=['price', 'quantity'], dtype=float)
bid_levels['side'] = 'bid'
bid_levels = bid_levels.sort_values('price', ascending=False)
bid_levels

,price,quantity,side
0,142.88,370.875,bid
1,142.87,285.057,bid
2,142.86,667.038,bid
3,142.85,644.689,bid
4,142.84,1478.048,bid
5,142.83,1477.730,bid
6,142.82,556.194,bid
7,142.81,253.645,bid
8,142.80,290.851,bid
9,142.79,378.691,bid


In [75]:
def calculate_mid_price(bid_levels, ask_levels):
    """
    Calculates the mid-price based on the best bid and best ask prices.

    The mid-price is the arithmetic mean of the highest bid price and the lowest ask price.
    It serves as a reference price for the asset, representing the center of the bid-ask spread.

    Args:
        bid_levels (pd.DataFrame): DataFrame containing bid levels with 'price' and 'quantity'.
                                   Expected to be sorted by price in descending order (best bid first).
        ask_levels (pd.DataFrame): DataFrame containing ask levels with 'price' and 'quantity'.
                                   Expected to be sorted by price in ascending order (best ask first).

    Returns:
        float: The calculated mid-price.
    """
    # Calculate the mid-price using the top-most levels (index 0)
    mid_price = (bid_levels['price'].iloc[0] + ask_levels['price'].iloc[0]) / 2.0
    return mid_price

In [76]:
def df_to_book(df, top_n=None):
    """
    Converts a pandas DataFrame of order book levels into a list of tuples.

    This utility function transforms the DataFrame representation of the order book
    into a list of (price, quantity) tuples. This format is often more efficient
    for sequential processing or iteration during simulation algorithms.

    Args:
        df (pd.DataFrame): The DataFrame containing order book levels. Must contain 'price' and 'quantity' columns.
        top_n (int, optional): The maximum number of top levels to include in the returned book.
                               If None, all levels in the DataFrame are included.

    Returns:
        list[tuple]: A list of tuples where each tuple is (price, quantity).
    """
    if top_n is None:
        # Use all rows if top_n is not specified
        rows = df.itertuples(index=False)
    else:
        # Limit to the top n rows
        rows = df.head(top_n).itertuples(index=False)
    
    # Create a list of (price, quantity) tuples
    book = [(row.price, row.quantity) for row in rows]
    return book

In [77]:
def execute_market_buy(ask_book, qty_to_buy):
    """
    Simulates the execution of a market buy order by walking the order book (LOB).

    This function models the matching engine's behavior for a market buy order. It consumes
    liquidity from the ask side of the order book, starting from the lowest price (best ask)
    and moving upwards until the desired quantity is filled. It calculates the volume-weighted
    average execution price and tracks the consumed liquidity.

    Args:
        ask_book (list[tuple]): A list of (price, quantity) tuples representing the ask side of the order book.
                                Expected to be sorted by price in ascending order.
        qty_to_buy (float): The total quantity of the asset to purchase.

    Returns:
        tuple: A tuple containing:
            - avg_exec_price (float or None): The volume-weighted average price at which the order was executed.
                                              Returns None if no quantity was executed.
            - executed_qty (float): The total quantity actually executed.
            - asks_after (list[tuple]): The state of the ask book after the order execution (remaining liquidity).
            - consumed_levels (list[tuple]): A list of (price, quantity) tuples representing the liquidity consumed.
    """
    # Create a copy of the book to avoid modifying the original list in place immediately
    asks = ask_book.copy()
    remaining_qty = qty_to_buy
    total_cost = 0.0
    consumed_levels = []

    # Iterate through the ask levels to fill the order
    for i, (price, qty) in enumerate(asks):
        if remaining_qty <= 0: 
            break

        qty_taken = min(qty, remaining_qty)
        total_cost += qty_taken * price
        remaining_qty -= qty_taken
        consumed_levels.append((price, qty_taken))
        asks[i] = (price, qty - qty_taken)
    
    asks_after = [(price, qty) for price, qty in asks if qty > 0]
    executed_qty = qty_to_buy - remaining_qty

    # Calculate the average execution price
    if executed_qty == 0:
        avg_exec_price = None
    else:
        avg_exec_price = total_cost / executed_qty
    return avg_exec_price, executed_qty, asks_after, consumed_levels

In [78]:
def run_simulation(bid_levels, ask_levels, sigma, eta, gamma, Q, top_n):
    """
    Runs a complete simulation of a market buy order to analyze slippage and market impact.

    This function orchestrates the simulation process. It prepares the order books, executes a market
    buy order against the ask book, and calculates key market microstructure metrics. Specifically,
    it computes the slippage (difference between the expected mid-price and the actual execution price)
    and estimates market impact components (temporary and permanent) based on theoretical models.

    Args:
        bid_levels (pd.DataFrame): DataFrame of bid levels (price, quantity).
        ask_levels (pd.DataFrame): DataFrame of ask levels (price, quantity).
        sigma (float): The asset volatility parameter, used in market impact models.
        eta (float): The temporary market impact coefficient.
        gamma (float): The permanent market impact coefficient.
        Q (float): The size of the order to execute (quantity).
        top_n (int): The number of order book levels to consider for the simulation.

    Returns:
        dict: A dictionary containing the detailed results of the simulation:
            - mid_before_price: The mid-price before the trade execution.
            - avg_exec_price_raw: The actual volume-weighted average execution price.
            - executed_quantity: The total quantity executed.
            - ask_levels_after: The state of the ask book after execution.
            - consumed_levels: The liquidity levels consumed during execution.
            - slippage_lob: The calculated slippage (Execution Price - Mid Price).
            - market_impact_temp: The estimated temporary market impact.
            - market_impact_perm: The estimated permanent market impact.
            - mid_after_price: The theoretical mid-price after the trade (incorporating permanent impact).
    """
    # Convert DataFrames to list of tuples for simulation efficiency
    bid_book = df_to_book(bid_levels, top_n)
    ask_book = df_to_book(ask_levels, top_n)

    # Calculate the initial mid-price (reference price)
    mid_before_price = calculate_mid_price(bid_levels, ask_levels)
    expected_price = mid_before_price

    # Execute the market buy order against the ask book
    actual_exec_avg_price_raw, executed_quantity, ask_levels_after, consumed_levels = execute_market_buy(ask_book, Q)
    
    if executed_quantity == 0:
        raise RuntimeError("No liquidity executed — either Q is 0 or book depth insufficient.")

    # Calculate Slippage: The difference between the actual execution price and the expected (mid) price
    slippage_lob = actual_exec_avg_price_raw - expected_price

    # Calculate Market Impact based on the Almgren-Chriss or similar impact models
    # Temporary Impact: Represents the cost of liquidity demand (sigma * eta * Q)
    market_impact_temp = sigma * eta * executed_quantity
    
    # Permanent Impact: Represents the information content of the trade (sigma * gamma * Q)
    market_impact_perm = sigma * gamma * executed_quantity

    # Estimate the new mid-price after the trade (incorporating permanent impact)
    mid_after = mid_before_price + market_impact_perm

    return {
        "mid_before_price": mid_before_price,
        "avg_exec_price_raw": actual_exec_avg_price_raw,
        "executed_quantity": executed_quantity,
        "ask_levels_after": ask_levels_after,
        "consumed_levels": consumed_levels,
        "slippage_lob": slippage_lob,
        "market_impact_temp": market_impact_temp,
        "market_impact_perm": market_impact_perm,
        "mid_after_price": mid_after
    }

In [79]:
if __name__ == "__main__":
    # --- Simulation Parameters ---
    sigma = 0.2   # Asset Volatility: Measure of the asset's price variation
    eta = 0.14    # Temporary Impact Coefficient: Factor for temporary price distortion
    gamma = 0.3   # Permanent Impact Coefficient: Factor for permanent price shift
    Q = 0.1       # Order Size: The quantity of the asset to buy
    
    top_n = 10    # Depth of the order book to consider

    # --- Run Simulation ---
    res = run_simulation(bid_levels, ask_levels, sigma, eta, gamma, Q, top_n)

    print("Simulation Results:")
    print(f"Mid-Price Before Execution: {res['mid_before_price']:.4f}")
    print(f"Average Execution Price:    {res['avg_exec_price_raw']:.4f}")
    print(f"Executed Quantity:          {res['executed_quantity']:.4f}")
    print(f"Slippage (LOB):             {res['slippage_lob']:.6f}")
    print(f"Market Impact (Temporary):  {res['market_impact_temp']:.6f}")
    print(f"Market Impact (Permanent):  {res['market_impact_perm']:.6f}")
    print(f"Mid-Price After Execution:  {res['mid_after_price']:.4f}")
    
    print(f"Ask Levels After: {res['ask_levels_after']}")
    print(f"Consumed Levels: {res['consumed_levels']}")

Simulation Results:
Mid-Price Before Execution: 142.8850
Average Execution Price:    142.8900
Executed Quantity:          0.1000
Slippage (LOB):             0.005000
Market Impact (Temporary):  0.002800
Market Impact (Permanent):  0.006000
Mid-Price After Execution:  142.8910
Ask Levels After: [(142.89, 122.325), (142.9, 667.183), (142.91, 289.495), (142.92, 141.42), (142.93, 82.802), (142.94, 160.841), (142.95, 124.375), (142.96, 108.832), (142.97, 152.221), (142.98, 238.425)]
Consumed Levels: [(142.89, 0.1)]
